In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
assert sklearn.__version__>= "0.20"
import gc
import seaborn as sns
from collections import Counter
import folium
import seaborn
import plotly.express as px

In [3]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,321 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123623 files and dire

In [4]:
# 한글 폰트 설정 (Mac과 Windows에 따라 폰트가 다를 수 있음)
plt.rc('font', family='NanumBarunGothic')

##### 1번 시각화 : 전주 대비 인원차이 (선 그래프)
 > 10월 7일, 10월 14일만(stay data)

In [5]:
df_1007 = pd.read_csv('/content/drive/MyDrive/빅데이터캡스톤디자인/stay_20231007_1.csv')
df_1006 = pd.read_csv('/content/drive/MyDrive/빅데이터캡스톤디자인/stay_20231006_1.csv')
df_1013 = pd.read_csv('/content/drive/MyDrive/빅데이터캡스톤디자인/stay_20231013_1.csv')
df_1014 = pd.read_csv('/content/drive/MyDrive/빅데이터캡스톤디자인/stay_20231014_1.csv')

In [6]:
df_1007.head(1)

,hdong_cd,date,time,gender,age,purpose,stay_cnts
0,4115056800,20231007,15:00,1,7,0,832


In [7]:
## 10월 13일
df13 = df_1013[df_1013['hdong_cd'] == 1111061500]
df13 = df13[df13['purpose'].isin([3, 5])]

In [8]:
## 10월 7일
df7 = df_1007[df_1007['hdong_cd'] == 1111061500]
df7 = df7[df7['purpose'].isin([3, 5])]
df7

,hdong_cd,date,time,gender,age,purpose,stay_cnts
135,1111061500,20231007,15:00,1,2,3,2623
1581,1111061500,20231007,16:00,1,2,3,2821
2408,1111061500,20231007,11:00,1,5,3,1294
5024,1111061500,20231007,22:00,0,0,5,847
9634,1111061500,20231007,15:00,0,4,3,1096
...,...,...,...,...,...,...,...
3716711,1111061500,20231007,21:00,1,8,3,16
3718659,1111061500,20231007,11:00,1,8,5,12
3728652,1111061500,20231007,21:00,1,8,5,12
3744240,1111061500,20231007,13:00,1,8,5,16


In [9]:
df7['stay_cnts'].sum()

336008

In [10]:
df14 = df_1014[df_1014['hdong_cd'] == 1111061500]
df14 = df14[df14['purpose'].isin([3, 5])]
df14

,hdong_cd,date,time,gender,age,purpose,stay_cnts
4323,1111061500,20231014,16:00,0,5,3,1134
5009,1111061500,20231014,12:00,1,7,3,966
7110,1111061500,20231014,11:00,1,0,3,1092
8988,1111061500,20231014,12:00,0,7,3,1044
11910,1111061500,20231014,15:00,0,4,3,900
...,...,...,...,...,...,...,...
3692568,1111061500,20231014,14:00,0,8,5,17
3707775,1111061500,20231014,18:00,0,8,5,11
3755701,1111061500,20231014,23:00,0,8,5,6
3800364,1111061500,20231014,21:00,1,8,3,8


In [11]:
df14['stay_cnts'].sum()

296289

In [12]:
# 'time' 열을 datetime의 시간 형식으로 변환
df7['time'] = pd.to_datetime(df7['time'], format='%H:%M').dt.time
df14['time'] = pd.to_datetime(df14['time'], format='%H:%M').dt.time

In [13]:
import plotly.express as px
import plotly.graph_objects as go

# 첫 번째 데이터 테이블에서 'time'별 'stay_cnts'의 합 계산
stay_cnts_sum1 = df7.groupby('time')['stay_cnts'].sum().reset_index()
stay_cnts_sum1['time'] = stay_cnts_sum1['time'].apply(lambda x: x.strftime('%H:%M'))

# 두 번째 데이터 테이블에서 'time'별 'stay_cnts'의 합 계산
stay_cnts_sum2 = df14.groupby('time')['stay_cnts'].sum().reset_index()
stay_cnts_sum2['time'] = stay_cnts_sum2['time'].apply(lambda x: x.strftime('%H:%M'))

# Plotly를 사용하여 두 날짜의 라인 그래프 생성
fig = go.Figure()

# 첫 번째 데이터 (10월 7일)
fig.add_trace(go.Scatter(x=stay_cnts_sum1['time'], y=stay_cnts_sum1['stay_cnts'],
                         mode='lines+markers', name='10월7일', line=dict(color='blue')))

# 두 번째 데이터 (10월 14일)
fig.add_trace(go.Scatter(x=stay_cnts_sum2['time'], y=stay_cnts_sum2['stay_cnts'],
                         mode='lines+markers', name='10월14일', line=dict(color='green')))

# 그래프 레이아웃 설정
fig.update_layout(
    title='10월 7일(축제 일주일 전)과 10월 14일(축제 당일) 체류인원 비교',
    xaxis_title='시간대',
    yaxis_title='체류인원(명)',
    xaxis=dict(tickmode='array'),
    yaxis=dict(tickformat=',',  # y축 숫자에 콤마 추가
               title='체류인원'),
    template='plotly_white',
    legend=dict(font=dict(size=25)),
    font=dict(size=28)
)

# 그래프 출력
fig.show()


#### 2. stay데이터 13,14 나이대별 체류인원

In [14]:
# age 값을 연령대로 변환하는 딕셔너리
age_mapping = {0: '10대 이하', 1: '10대', 2: '20대', 3: '30대', 4: '40대',
               5: '50대', 6: '60대', 7: '70대', 8: '80대 이상'}

# age 컬럼 값을 연령대로 변환
df13['age'] = df13['age'].replace(age_mapping)
# age 컬럼 값을 연령대로 변환
df14['age'] = df14['age'].replace(age_mapping)

# age 컬럼을 순서가 있는 범주형으로 변환
age_categories = ['10대 이하', '10대', '20대', '30대', '40대', '50대', '60대', '70대', '80대 이상']
df13['age'] = pd.Categorical(df13['age'], categories=age_categories, ordered=True)
df14['age'] = pd.Categorical(df14['age'], categories=age_categories, ordered=True)

In [15]:
# 'age'별 'stay_cnts'의 합 계산
age_stay_cnts_sum13= df13.groupby('age')['stay_cnts'].sum().reset_index()
age_stay_cnts_sum14= df14.groupby('age')['stay_cnts'].sum().reset_index()

<ipython-input-15-3cb6f6bbec85>:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

<ipython-input-15-3cb6f6bbec85>:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [16]:
### 10월 13일 나이대별 체류인원
import plotly.graph_objects as go

# Plotly 막대 그래프 생성
fig = go.Figure()

# 막대 그래프 추가
fig.add_trace(go.Bar(
    x=age_stay_cnts_sum13['age'],  # x축: 나이대
    y=age_stay_cnts_sum13['stay_cnts'],  # y축: 체류인원
    text=age_stay_cnts_sum13['stay_cnts'],  # 막대 위에 표시할 값
    textposition='outside',  # 막대 위에 값 표시
    marker_color='skyblue',  # 막대 색상
    texttemplate='%{text:,}'  # 콤마 추가된 숫자 형식
))

# 그래프 레이아웃 설정
fig.update_layout(
    title='10월 13일 금요일 나이대별 체류인원',
    xaxis_title='나이대',
    yaxis_title='체류인원',
    xaxis=dict(tickmode='array'),
    yaxis=dict(tickformat=',',  # y축 숫자에 콤마 추가
               title='체류인원(명)'),
    template='plotly_white',
    font=dict(size=15)
)

# 그래프 출력
fig.show()


In [17]:
### 10월 14일 나이대별 체류인원
import plotly.graph_objects as go

# Plotly 막대 그래프 생성
fig = go.Figure()

# 막대 그래프 추가
fig.add_trace(go.Bar(
    x=age_stay_cnts_sum14['age'],  # x축: 나이대
    y=age_stay_cnts_sum14['stay_cnts'],  # y축: 체류인원
    text=age_stay_cnts_sum14['stay_cnts'],  # 막대 위에 표시할 값
    textposition='outside',  # 막대 위에 값 표시
    marker_color='skyblue',  # 막대 색상
    texttemplate='%{text:,}'  # 콤마 추가된 숫자 형식
))

# 그래프 레이아웃 설정
fig.update_layout(
    title='10월 14일 토요일 나이대별 체류인원',
    xaxis_title='나이대',
    yaxis_title='체류인원',
    xaxis=dict(tickmode='array'),
    yaxis=dict(tickformat=',',  # y축 숫자에 콤마 추가
               title='체류인원(명)'),
    template='plotly_white',
    legend=dict(font=dict(size=15)),
    font=dict(size=15)
)

# 그래프 출력
fig.show()

In [18]:
### 10월 14일 나이대별 체류인원
import plotly.graph_objects as go

# age 값을 연령대로 변환하는 딕셔너리
gender= {0: '남성', 1: '여성'}

# age 컬럼 값을 연령대로 변환
df13['gender'] = df13['gender'].replace(gender)
# age 컬럼 값을 연령대로 변환
df14['gender'] = df14['gender'].replace(gender)

gender_13= df13.groupby('gender')['stay_cnts'].sum().reset_index()
gender_14= df14.groupby('gender')['stay_cnts'].sum().reset_index()


# Plotly 막대 그래프 생성
fig = go.Figure()

# 막대 그래프 추가
fig.add_trace(go.Bar(
    x=gender_13['gender'],  # x축: 나이대
    y=gender_13['stay_cnts'],  # y축: 체류인원
    text=gender_13['stay_cnts'],  # 막대 위에 표시할 값
    textposition='outside',  # 막대 위에 값 표시
    marker_color='skyblue',  # 막대 색상
    texttemplate='%{text:,}'  # 콤마 추가된 숫자 형식
))

# 그래프 레이아웃 설정
fig.update_layout(
    title='10월 13일 금요일 성별 분포',
    xaxis_title='성별',
    yaxis_title='체류인원',
    xaxis=dict(tickmode='array'),
    yaxis=dict(tickformat=',',  # y축 숫자에 콤마 추가
               title='체류인원(명)'),
    template='plotly_white',
    legend=dict(font=dict(size=15)),
    font=dict(size=18)
)

# 그래프 출력
fig.show()

In [19]:
### 10월 14일 나이대별 체류인원
import plotly.graph_objects as go

# age 값을 연령대로 변환하는 딕셔너리
gender= {0: '남성', 1: '여성'}

# age 컬럼 값을 연령대로 변환
df13['gender'] = df13['gender'].replace(gender)
# age 컬럼 값을 연령대로 변환
df14['gender'] = df14['gender'].replace(gender)

gender_13= df13.groupby('gender')['stay_cnts'].sum().reset_index()
gender_14= df14.groupby('gender')['stay_cnts'].sum().reset_index()


# Plotly 막대 그래프 생성
fig = go.Figure()

# 막대 그래프 추가
fig.add_trace(go.Bar(
    x=gender_14['gender'],  # x축: 나이대
    y=gender_14['stay_cnts'],  # y축: 체류인원
    text=gender_14['stay_cnts'],  # 막대 위에 표시할 값
    textposition='outside',  # 막대 위에 값 표시
    marker_color='skyblue',  # 막대 색상
    texttemplate='%{text:,}'  # 콤마 추가된 숫자 형식
))

# 그래프 레이아웃 설정
fig.update_layout(
    title='10월 14일 토요일 성별 분포',
    xaxis_title='성별',
    yaxis_title='체류인원',
    xaxis=dict(tickmode='array'),
    yaxis=dict(tickformat=',',  # y축 숫자에 콤마 추가
               title='체류인원(명)'),
    template='plotly_white',
    legend=dict(font=dict(size=15)),
    font=dict(size=15)
)

# 그래프 출력
fig.show()

In [20]:
day1_map = pd.read_csv('/content/drive/MyDrive/빅데이터캡스톤디자인/df_1013.csv')
day1_map.head(3)

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts,행정동코드,시도명,시군구명,읍면동명,주소,위도,경도
0,1114060500,1111061500,20231013,17:00,17:00,1,1,2.0,3.0,3,39079,54,7,1114060500,서울특별시,중구,을지로동,서울특별시 중구 을지로동,37.566990,126.991194
1,4146356000,1111061500,20231013,12:00,13:00,1,1,2.0,0.0,3,53748,90,7,4146356000,경기도,용인시 기흥구,구성동,경기도 용인시 기흥구 구성동,37.296759,127.118988
2,1111068000,1111061500,20231013,16:00,16:00,1,1,0.0,5.0,5,6434,21,7,1111068000,서울특별시,종로구,창신제2동,서울특별시 종로구 창신제2동,37.574482,127.010872


In [21]:
day2_map = pd.read_csv('/content/drive/MyDrive/빅데이터캡스톤디자인/df_1014.csv')
day2_map.head(1)

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts,행정동코드,시도명,시군구명,읍면동명,주소,위도,경도
0,1114055000,1111061500,20231014,12:00,13:00,1,0,0.0,4.0,4,9500,29,44,1114055000,서울특별시,중구,명동,서울특별시 중구 명동,37.558358,126.985242


In [22]:
## 쇼핑여가, 여행만 뽑아내기
day1 = day1_map[(day1_map['dest_purpose']==3) | (day1_map['dest_purpose']==5)]
day1.head(1)

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts,행정동코드,시도명,시군구명,읍면동명,주소,위도,경도
0,1114060500,1111061500,20231013,17:00,17:00,1,1,2.0,3.0,3,39079,54,7,1114060500,서울특별시,중구,을지로동,서울특별시 중구 을지로동,37.56699,126.991194


In [23]:
## 쇼핑여가, 여행만 뽑아내기
day2 = day2_map[(day2_map['dest_purpose']==3) | (day2_map['dest_purpose']==5)]
day2.head(1)

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts,행정동코드,시도명,시군구명,읍면동명,주소,위도,경도
1,1111053000,1111061500,20231014,10:00,10:00,1,1,4.0,3.0,3,232,1,7,1111053000,서울특별시,종로구,사직동,서울특별시 종로구 사직동,37.575422,126.96632


In [24]:
day1.shape

(1512, 20)

In [25]:
day1['시도명'].value_counts().sum()

1512

In [26]:
### 가장 많이 방문하는 지역 파악 : 서울특별시, 경기도, 인천광역시와 같은 서울 주변 지역에서 가장 많이 방문함
day1['시도명'].value_counts()

,count
시도명,
서울특별시,1271
경기도,181
인천광역시,19
강원특별자치도,15
충청남도,7
충청북도,5
대구광역시,5
경상남도,3
부산광역시,2


In [27]:
## 평균 이동시간은 49분 정도
day1['od_duration_avg'].mean()

49.79960317460318

In [28]:
day2['시도명'].value_counts().sum()

2172

In [29]:
day2['시도명'].value_counts()


,count
시도명,
서울특별시,1804
경기도,293
인천광역시,33
충청남도,15
대전광역시,6
충청북도,5
강원특별자치도,5
대구광역시,3
부산광역시,2


#### 4. 주 이동수단 시각화

In [30]:
od1 = pd.read_csv('/content/drive/MyDrive/빅데이터캡스톤디자인/df_1013.csv')
od2 = pd.read_csv('/content/drive/MyDrive/빅데이터캡스톤디자인/df_1014.csv')
## 쇼핑여가, 여행만 뽑아내기
od1 = od1[(od1['dest_purpose']==3) | (od1['dest_purpose']==5)]
od2 = od2[(od2['dest_purpose']==3) | (od2['dest_purpose']==5)]
od1.shape

(1512, 20)

In [31]:
od2.shape

(2172, 20)

In [32]:
od1.head(2)

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts,행정동코드,시도명,시군구명,읍면동명,주소,위도,경도
0,1114060500,1111061500,20231013,17:00,17:00,1,1,2.0,3.0,3,39079,54,7,1114060500,서울특별시,중구,을지로동,서울특별시 중구 을지로동,37.566990,126.991194
1,4146356000,1111061500,20231013,12:00,13:00,1,1,2.0,0.0,3,53748,90,7,4146356000,경기도,용인시 기흥구,구성동,경기도 용인시 기흥구 구성동,37.296759,127.118988


In [33]:
# age 값을 연령대로 변환하는 딕셔너리
mapping = {0: '차량', 1: '시내버스', 2: '지하철', 3: '도보', 4: '기타',
               5: '철도', 6: '시외고속버스', 7: '항공기'}

# age 컬럼 값을 연령대로 변환
od1['modal'] = od1['modal'].replace(mapping)
# age 컬럼 값을 연령대로 변환
od2['modal'] = od2['modal'].replace(mapping)

In [34]:
##10월 13일 이동수단

import plotly.graph_objects as go

# 이동수단별 데이터 내림차순으로 정렬
modal_cnt1= od1.groupby('modal')['od_cnts'].sum().reset_index()
modal_cnt1 = modal_cnt1.sort_values(by='od_cnts', ascending=False).reset_index(drop=True)

# Plotly 막대 그래프 생성
fig = go.Figure()

# 막대 그래프 추가
fig.add_trace(go.Bar(
    x=modal_cnt1['modal'],  # x축: 이동수단
    y=modal_cnt1['od_cnts'],  # y축: 이동인원
    text=modal_cnt1['od_cnts'],  # 막대 위에 표시할 값
    textposition='outside',  # 막대 위에 값 표시
    marker_color='skyblue',  # 막대 색상
    texttemplate='%{text:,}'  # 콤마 추가된 숫자 형식
))

# 그래프 레이아웃 설정
fig.update_layout(
    title='10월 13일 금요일 이동수단 분포',
    xaxis_title='이동수단',
    yaxis_title='이동인원',
    xaxis=dict(tickmode='array'),
    yaxis=dict(tickformat=',',  # y축 숫자에 콤마 추가
               title='이동인원(명)'),
    template='plotly_white',
    legend=dict(font=dict(size=15)),
    font=dict(size=15)
)

# 그래프 출력
fig.show()


In [35]:
import plotly.graph_objects as go

# 이동수단별 데이터 내림차순으로 정렬
modal_cnt2 = od2.groupby('modal')['od_cnts'].sum().reset_index()
modal_cnt2 = modal_cnt2.sort_values(by='od_cnts', ascending=False).reset_index(drop=True)

# Plotly 막대 그래프 생성
fig = go.Figure()

# 막대 그래프 추가
fig.add_trace(go.Bar(
    x=modal_cnt2['modal'],  # x축: 이동수단
    y=modal_cnt2['od_cnts'],  # y축: 이동인원
    text=modal_cnt2['od_cnts'],  # 막대 위에 표시할 값
    textposition='outside',  # 막대 위에 값 표시
    marker_color='skyblue',  # 막대 색상
    texttemplate='%{text:,}'  # 콤마 추가된 숫자 형식
))

# 그래프 레이아웃 설정
fig.update_layout(
    title='10월 14일 토요일 이동수단 분포',
    xaxis_title='이동수단',
    yaxis_title='이동인원',
    xaxis=dict(tickmode='array'),
    yaxis=dict(tickformat=',',  # y축 숫자에 콤마 추가
               title='이동인원(명)'),
    template='plotly_white',
    legend=dict(font=dict(size=15)),
    font=dict(size=15)
)

# 그래프 출력
fig.show()

### 1성별 분포

In [36]:
# age 값을 연령대로 변환하는 딕셔너리
map = {0: '남성', 1: '여성'}

# age 컬럼 값을 연령대로 변환
od1['gender'] = od1['gender'].replace(map)
# age 컬럼 값을 연령대로 변환
od2['gender'] = od2['gender'].replace(map)

In [37]:
!pip install pydeck

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.6 MB/s eta 0:00:00


In [38]:
## 서울 데이터만 뽑아내기
seoul = day1[day1['시도명'] == '서울특별시']
seoul

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts,행정동코드,시도명,시군구명,읍면동명,주소,위도,경도
0,1114060500,1111061500,20231013,17:00,17:00,1,1,2.0,3.0,3,39079,54,7,1114060500,서울특별시,중구,을지로동,서울특별시 중구 을지로동,37.566990,126.991194
2,1111068000,1111061500,20231013,16:00,16:00,1,1,0.0,5.0,5,6434,21,7,1111068000,서울특별시,종로구,창신제2동,서울특별시 종로구 창신제2동,37.574482,127.010872
5,1144068000,1111061500,20231013,09:00,09:00,1,0,0.0,5.0,5,18562,40,44,1144068000,서울특별시,마포구,합정동,서울특별시 마포구 합정동,37.551662,126.911756
13,1117052000,1111061500,20231013,22:00,22:00,1,1,0.0,5.0,5,15891,30,7,1117052000,서울특별시,용산구,용산2가동,서울특별시 용산구 용산2가동,37.546126,126.985574
19,1165062100,1111061500,20231013,16:00,19:00,1,1,2.0,0.0,3,77009,150,7,1165062100,서울특별시,서초구,방배4동,서울특별시 서초구 방배4동,37.489060,126.992483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3453,1114052000,1111061500,20231013,21:00,21:00,1,1,0.0,3.0,3,9558,16,7,1114052000,서울특별시,중구,소공동,서울특별시 중구 소공동,37.563900,126.979794
3455,1114060500,1111061500,20231013,17:00,17:00,0,7,4.0,3.0,3,593,4,6,1114060500,서울특별시,중구,을지로동,서울특별시 중구 을지로동,37.566990,126.991194
3461,1111054000,1111061500,20231013,19:00,19:00,1,2,4.0,3.0,3,1001,6,5,1111054000,서울특별시,종로구,삼청동,서울특별시 종로구 삼청동,37.584998,126.981758
3465,1114060500,1111061500,20231013,13:00,13:00,1,2,0.0,3.0,3,11239,21,5,1114060500,서울특별시,중구,을지로동,서울특별시 중구 을지로동,37.566990,126.991194


In [39]:
## 10월 13일(서울만)
import pydeck as pdk
# pydeck ArcLayer 설정
layer = pdk.Layer(
    "ArcLayer",
    seoul,
    get_source_position="[경도, 위도]",  # 출발지 좌표
    get_target_position="[126.990287260844, 37.5744355396607]",      # 도착지 좌표(종로구 종로1,2,3,4가동 좌표)
    get_width=4,  # Arc 두께
    get_source_color=[255, 255, 120],  # 출발지 색상
    get_target_color=[255, 0, 0],      # 도착지 색상
    pickable=True,  # 마우스 클릭 가능 여부
    auto_highlight=True  # 마우스 하이라이트 여부
)

# view_state 설정 (지도의 중심 및 시각화 각도 등)
view_state = pdk.ViewState(
    latitude= seoul['위도'].mean(),  # 평균 위도를 중심으로 설정
    longitude=seoul['경도'].mean(),  # 평균 경도를 중심으로 설정
    zoom=10,  # 줌 레벨
    bearing=-15,  # 지도의 회전각도
    pitch=45  # 지도의 기울기
)

# Deck 생성 및 ArcLayer 추가
r = pdk.Deck(layers=[layer], initial_view_state=view_state)

# 결과 지도 보기
r.show()  # Jupyter 노트북 환경에서는 지도 렌더링, 웹 브라우저로 출력
# 지도 저장



<IPython.core.display.Javascript object>

In [40]:
nseoul = day1[day1['시도명'] != '서울특별시']
nseoul.info()

<class 'pandas.core.frame.DataFrame'>
Index: 241 entries, 1 to 3397
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   origin_hdong_cd  241 non-null    int64  
 1   dest_hdong_cd    241 non-null    int64  
 2   date             241 non-null    int64  
 3   start_time       241 non-null    object 
 4   end_time         241 non-null    object 
 5   gender           241 non-null    int64  
 6   age              241 non-null    int64  
 7   modal            241 non-null    float64
 8   origin_purpose   241 non-null    float64
 9   dest_purpose     241 non-null    int64  
 10  od_dist_avg      241 non-null    int64  
 11  od_duration_avg  241 non-null    int64  
 12  od_cnts          241 non-null    int64  
 13  행정동코드            241 non-null    int64  
 14  시도명              241 non-null    object 
 15  시군구명             241 non-null    object 
 16  읍면동명             241 non-null    object 
 17  주소               241

In [41]:
## 10월 13일(서울이 아닌 지역)
import pydeck as pdk
# pydeck ArcLayer 설정
layer = pdk.Layer(
    "ArcLayer",
    nseoul,
    get_source_position="[경도, 위도]",  # 출발지 좌표
    get_target_position="[126.990287260844, 37.5744355396607]",      # 도착지 좌표(종로구 종로1,2,3,4가동 좌표)
    get_width=4,  # Arc 두께
    get_source_color=[255, 255, 120],  # 출발지 색상
    get_target_color=[255, 0, 0],      # 도착지 색상
    pickable=True,  # 마우스 클릭 가능 여부
    auto_highlight=True  # 마우스 하이라이트 여부
)

# view_state 설정 (지도의 중심 및 시각화 각도 등)
view_state = pdk.ViewState(
    latitude= nseoul['위도'].mean(),  # 평균 위도를 중심으로 설정
    longitude=nseoul['경도'].mean(),  # 평균 경도를 중심으로 설정
    zoom=10,  # 줌 레벨
    bearing=-15,  # 지도의 회전각도
    pitch=45  # 지도의 기울기
)

# Deck 생성 및 ArcLayer 추가
r = pdk.Deck(layers=[layer], initial_view_state=view_state)

# 결과 지도 보기
r.show()  # Jupyter 노트북 환경에서는 지도 렌더링, 웹 브라우저로 출력
# 지도 저장


<IPython.core.display.Javascript object>

In [42]:
seoul2 = day2[day2['시도명'] == '서울특별시']
nseoul2 = day2[day2['시도명'] != '서울특별시']

In [43]:
## 10월 14일(서울만)
import pydeck as pdk
# pydeck ArcLayer 설정
layer = pdk.Layer(
    "ArcLayer",
    seoul2,
    get_source_position="[경도, 위도]",  # 출발지 좌표
    get_target_position="[126.990287260844, 37.5744355396607]",      # 도착지 좌표(종로구 종로1,2,3,4가동 좌표)
    get_width=4,  # Arc 두께
    get_source_color=[255, 255, 120],  # 출발지 색상
    get_target_color=[255, 0, 0],      # 도착지 색상
    pickable=True,  # 마우스 클릭 가능 여부
    auto_highlight=True  # 마우스 하이라이트 여부
)

# view_state 설정 (지도의 중심 및 시각화 각도 등)
view_state = pdk.ViewState(
    latitude= seoul2['위도'].mean(),  # 평균 위도를 중심으로 설정
    longitude=seoul2['경도'].mean(),  # 평균 경도를 중심으로 설정
    zoom=10,  # 줌 레벨
    bearing=-15,  # 지도의 회전각도
    pitch=45  # 지도의 기울기
)

# Deck 생성 및 ArcLayer 추가
r = pdk.Deck(layers=[layer], initial_view_state=view_state)

# 결과 지도 보기
r.show()  # Jupyter 노트북 환경에서는 지도 렌더링, 웹 브라우저로 출력

<IPython.core.display.Javascript object>

In [44]:
## 10월 14일(서울이 아닌 지역)
import pydeck as pdk
# pydeck ArcLayer 설정
layer = pdk.Layer(
    "ArcLayer",
    nseoul2,
    get_source_position="[경도, 위도]",  # 출발지 좌표
    get_target_position="[126.990287260844, 37.5744355396607]",      # 도착지 좌표(종로구 종로1,2,3,4가동 좌표)
    get_width=4,  # Arc 두께
    get_source_color=[255, 255, 120],  # 출발지 색상
    get_target_color=[255, 0, 0],      # 도착지 색상
    pickable=True,  # 마우스 클릭 가능 여부
    auto_highlight=True  # 마우스 하이라이트 여부
)

# view_state 설정 (지도의 중심 및 시각화 각도 등)
view_state = pdk.ViewState(
    latitude= nseoul2['위도'].mean(),  # 평균 위도를 중심으로 설정
    longitude=nseoul2['경도'].mean(),  # 평균 경도를 중심으로 설정
    zoom=10,  # 줌 레벨
    bearing=-15,  # 지도의 회전각도
    pitch=45  # 지도의 기울기
)

# Deck 생성 및 ArcLayer 추가
r = pdk.Deck(layers=[layer], initial_view_state=view_state)

# 결과 지도 보기
r.show()  # Jupyter 노트북 환경에서는 지도 렌더링, 웹 브라우저로 출력

<IPython.core.display.Javascript object>

In [45]:
day1_map['도착지_위도'] = 37.5744355396607
day1_map['도착지_경도'] = 126.990287260844

day2_map['도착지_위도'] = 37.5744355396607
day2_map['도착지_경도'] = 126.990287260844

day1_map.to_csv('day1_final.csv', index = False)
day2_map.to_csv('day2_final.csv', index = False)

##### 정리 및 결론
1. 주 이동수단은 자동차와 지하철임(택시가 포함되어 있음)
2. 대부분의 사람들이 서울에서 가장 많이 방문하고 다음이 경기도와 인천임
3. 주 방문 시간은 점심시간대와 오후 시간대임
4. 주 방문층은 10대가 가장 많이 방문함
5. 평일에는 10대가 주말에는 10대이하가 가장 많이 방문함
6. 평일보다 주말이 평균 이동시간이 긴 편

위와 같은 패턴으로 보았을 때 방문객 마케팅 전략에 관한 것이 주제로 적당해보임

* 주말에 10대이하와 10대가 가장 많이 방문하는 연령대인 것을 고려해 공연 관람보다 체험형 프로그램 기획 마련
ex) 악기 체험 혹은 서울 돈화문국악당이 배경인만큼 한복 대여를 가능하게 해 인생샷 남기기와 같은 참여 독려 이벤트 열기
* 축제 전부터 sns의 숏폼 제작 홍보 이벤트를 열어 국악축제를 잘 홍보해주는 영상을 제작하게 하여 상품을 걸어 사람들의 관심과 참여를 이끌어내는 것
* 데이터와를 보면 점심시간과 오후시간대에 사람이 많이 방문하는 것을 확인할 수 있음, 출출한 시간대이므로 간식거리가 될 수 있는 푸드트럭 혹은 주변 맛집 관련 지도와 같은 홈페이지를 제공
* 데이터에는 없지만 외국인 관광객을 타겟으로 해서 지하철을 이용하는 외국인들한테 종로 가까이에서 국악축제가 있다는 것을 방송으로 광고형태로 전하기